In [1]:
import pandas as pd
import requests
import csv
import numpy as np

class IntradayDataFetcher:
    def __init__(self):
        self.data = {}

    def fetch_intraday_data(self, script):
        CSV_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+script+'&interval=15min&slice=year1month1&apikey=TFANB8QLXP6NDVRB'

        with requests.Session() as s:
            download = s.get(CSV_URL)
            decoded_content = download.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            my_list = list(cr)
            self.data[script] = my_list[1:]  # Exclude header row

    def convert_intraday_data(self, script):
        if script in self.data:
            intraday_data = self.data[script]
            df = pd.DataFrame(intraday_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
            return df
        else:
            print(f"No intraday data found for script: {script}")

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __contains__(self, key):
        return key in self.data


def indicator1(df, timeperiod):
    indicator_values = df['close'].rolling(window=timeperiod).mean()
    indicator_df = pd.DataFrame({'timestamp': df['timestamp'], 'indicator': indicator_values})
    return indicator_df


class Strategy:
    def __init__(self):
        self.data_fetcher = IntradayDataFetcher()

    def generate_signals(self, script):
        self.data_fetcher.fetch_intraday_data(script)
        df = self.data_fetcher.convert_intraday_data(script)
        indicator_data = indicator1(df, timeperiod=5)
        
        signals = pd.DataFrame({'timestamp': df['timestamp'], 'signal': np.nan})

        for i in range(1, len(df)):
            if indicator_data['indicator'].iloc[i] > df['close'].iloc[i]:
                if indicator_data['indicator'].iloc[i-1] <= df['close'].iloc[i-1]:
                    signals.loc[i, 'signal'] = 'BUY'
                else:
                    signals.loc[i, 'signal'] = 'NO_SIGNAL'
            elif indicator_data['indicator'].iloc[i] < df['close'].iloc[i]:
                if indicator_data['indicator'].iloc[i-1] >= df['close'].iloc[i-1]:
                    signals.loc[i, 'signal'] = 'SELL'
                else:
                    signals.loc[i, 'signal'] = 'NO_SIGNAL'
            else:
                signals.loc[i, 'signal'] = 'NO_SIGNAL'

        return signals


# Usage example:
strategy = Strategy()
script = 'GOOGL'
signals = strategy.generate_signals(script)
print(signals)


               timestamp     signal
0    2023-06-13 19:45:00        NaN
1    2023-06-13 19:30:00  NO_SIGNAL
2    2023-06-13 19:15:00  NO_SIGNAL
3    2023-06-13 19:00:00  NO_SIGNAL
4    2023-06-13 18:45:00  NO_SIGNAL
...                  ...        ...
1309 2023-05-15 05:00:00  NO_SIGNAL
1310 2023-05-15 04:45:00  NO_SIGNAL
1311 2023-05-15 04:30:00  NO_SIGNAL
1312 2023-05-15 04:15:00        BUY
1313 2023-05-15 04:00:00  NO_SIGNAL

[1314 rows x 2 columns]


In [2]:
filtered_signals = signals[signals['signal'].isin(['BUY', 'SELL'])]
print(filtered_signals)

               timestamp signal
5    2023-06-13 18:30:00    BUY
8    2023-06-13 17:45:00   SELL
10   2023-06-13 17:15:00    BUY
15   2023-06-13 16:00:00   SELL
17   2023-06-13 15:30:00    BUY
...                  ...    ...
1292 2023-05-15 09:15:00    BUY
1296 2023-05-15 08:15:00   SELL
1302 2023-05-15 06:45:00    BUY
1306 2023-05-15 05:45:00   SELL
1312 2023-05-15 04:15:00    BUY

[384 rows x 2 columns]
